In [451]:
import requests
import bs4
import difflib
import time
import sqlite3
c = sqlite3.connect('TOS.sqlite')

c.execute("""
CREATE TABLE IF NOT EXISTS company(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    url TEXT,
    name TEXT,
    last_scan INT,
    last_error INT,
    scan_instructions TEXT,
    status TEXT
)
""")

c.execute("""
CREATE TABLE IF NOT EXISTS tos_text(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    company_id INT,
    start_date INT,
    end_date INT,
    text BLOB,
    formatted_text BLOB,
    delta BLOB,
    formatted_delta BLOB
)
""")

c.execute("""
CREATE INDEX IF NOT EXISTS tos_text_company_idx ON tos_text(company_id)
""")
c.commit()


def iterative_diff_function(a, b):
    difflib._check_types(a, b, '','','','','\n')
    for tag, i1, i2, j1, j2 in difflib.SequenceMatcher(lambda x: x==' ',a,b).get_opcodes():
        if tag == "equal":
            yield (tag, a[i1:i2])
        if tag == "delete":
            yield (tag, a[i1:i2])
        if tag == "insert":
            yield (tag, b[j1:j2])
        if tag == "replace":
            yield (tag, a[i1:i2], b[j1:j2])

            
def diff_function(a, b):
    return list(iterative_diff_function(a, b))


_diff_function = diff_function


def do_update_check(company_id):
    old_tos = lookup_TOS(company_id)
    url = lookup_URL(company_id)
    new_tos = pull_TOS(url)
    if old_tos:
        text_diff = list(_diff_function(
            old_tos['text'],
            new_tos['text'],
        ))
        format_diff = list(_diff_function(
            old_tos['formatted_text'],
            new_tos['formatted_text'],
        ))
        if len(text_diff) == len(format_diff) == 1:
            if (text_diff[0][0] == format_diff[0][0] == "equal"):
                return None
        return {
            "text": new_tos["text"],
            "formatted_text": new_tos["formatted_text"],
            "delta": text_diff,
            "formatted_delta": format_diff,
            "new": False,
        }
    else:
        return {
            "text": new_tos["text"],
            "formatted_text": new_tos["formatted_text"],
            "new": True
        }

    
def pull_TOS(url):
    r = requests.get(url, timeout=10)
    assert r.status_code == 200, 'error, code: ' + r.status_code
    soup = bs4.BeautifulSoup(r.text)
    return {
        "text": soup.body.text,  #"\n\n".join(p.text for p in ps),
        "formatted_text": str(soup.body),     #"\n\n".join(str(p) for p in ps),
    }


def create_company(name, url, settings):
    c.execute('INSERT INTO company(name, url) VALUES (?,?)', (name, url))
    c.commit()

    
def update_company_name(id, name=None):
    c.execute('UPDATE company SET name=? WHERE id=?', (name, id))
    c.commit()

    
def update_company_url(id, url=None):
    c.execute('UPDATE company SET url=? WHERE id=?', (name, id))
    c.commit()

    
def update_last_scan(id, last_scan=None):
    c.execute('UPDATE company SET last_scan WHERE id=?', (last_scan, id))
    c.commit()

    
def update_last_error(company_id, error, dt):
    c.commit();

    
def add_TOS(company_id, text, formatted_text, delta, formatted_delta, current_time):
    last_tos = c.execute(
        """SELECT id FROM tos_text WHERE company_id=? ORDER BY start_date DESC""", 
        (company_id, )
    ).fetchone()
    if last_tos:
        c.execute(
            """UPDATE tos_text SET end_date=? WHERE id=?""", 
            (start_date, last_tos[0],)
        )
        c.commit()
    c.execute(
        """
        INSERT INTO tos_text(company_id, start_date, text, formatted_text, delta, formatted_delta) 
        VALUES (?,?,?,?,?,?)
        """, 
        (
            company_id, 
            start_date, 
            text, 
            formatted_text, 
            json.dumps(delta), 
            json.dumps(formatted_delta)
        )
    )
    c.commit()

    
def lookup_URL(company_id):
    return c.execute("select url from company where id=?", (company_id,)).fetchone()[0]


def lookup_TOS(company_id):
    last_tos = c.execute("""
        select text, formatted_text from tos_text where company_id=? 
        order by start_date desc
        """, (company_id,)
    ).fetchone()
    if not last_tos:
        return None
    else:
        return {
            "text": last_tos[0],
            "formatted_text": last_tos[1],
        }
        

def scan_company_tos(company_id):
    url = lookup_URL(company_id)
    tos = pull_TOS(url)
    lookup_TOS(company_id)
    update_result = do_update_check(company_id)

    if not len(update_result.get("delta", [])) > 1:
        print("no change")
        pass # nullop
    else:
        add_TOS(
            company_id, 
            update_result["text"], 
            update_result["formatted_text"], 
            update_result.get("delta"),
            update_result.get("formatted_delta"),
            int(time.time())
       )
    # TODO: log error and update status to show most recent error


In [452]:
if False:
    create_company("_local test", "http://0.0.0.0:8999/www/", {})
    print('\n'.join(c.execute("select * from company limit 20;")))

# Pages

## Overview

- String Filter
- Time Filter

- Table of companies
    + Name
    + url
    + Last update
    + number of changes by year
    + edit button


## Main Changes View

- String Filter
- Start time scrubber
- End time scrubber

- List of changes by company
    - left / right, just changes and surroundings


## Company page

- select left pane
- select right pane
- go to company setting page


## Company settings

- change URL
- change monitored portion


## Add Company page

- name
- url
- test page
